<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Prompt_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://withpi.ai/logoFullBlack.svg" width="240"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://play.withpi.ai"><font size="4">Technique Catalog</font></a>

# Prompt Optimization Technique


This colab assumes that you already went through [Input Generation](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Input_Generation.ipynb), and now wish to optimize your system prompt

We will walk through the same `Aesop AI` example, but you can load any contract here. Let's dig in!

This should take about **15 minutes**, even if you're unfamiliar with Colab.

## Install and initialize SDK

Connect to a regular CPU Python 3 runtime.  You won't need GPUs for this notebook.

You'll need a WITHPI_API_KEY from https://play.withpi.ai.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [ ]:
%%capture

%pip install withpi httpx pandas tqdm litellm jinja2

import os
from google.colab import userdata
from withpi import PiClient

os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

client = PiClient()

# Load contract and Dataset

Load the `Aesop AI` example and example set from Pi Labs cookbooks, or edit below to load a different one.


In [ ]:
import httpx
import pandas as pd
from google.colab import data_table
from withpi.types import Contract

resp = httpx.get("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/contracts/aesop_ai.json")

aesop_contract = Contract.model_validate_json(resp.content)

for dimension in aesop_contract.dimensions:
  print(dimension.label)
  for sub_dimension in dimension.sub_dimensions:
    print(f"\t{sub_dimension.description}")

df = pd.read_parquet("https://raw.githubusercontent.com/withpi/cookbook-withpi/refs/heads/main/datasets/aesop_ai_examples.parquet")
data_table.enable_dataframe_formatter()
df


## Optimize your prompt

Kick off a prompt optimization run.  This will operate in the background.

In [ ]:
prompt_optimization_status = client.prompt.optimize(
    contract=aesop_contract,
    dspy_optimization_type="COPRO",
    examples=[{"llm_input": row["input"], "llm_output": row["output"]} for index, row in df.iterrows()],
    initial_system_instruction=aesop_contract.description,
    model_id="gpt-4o-mini",
    tuning_algorithm="DSPY",
)

## Check for completion

The following cell will connect to the tail of logs while optimization proceeds.  It will take order of **10 minutes**

In [ ]:
import json

while True:
  optimized_response = client.prompt.get_status(job_id=prompt_optimization_status.job_id)
  if (optimized_response.state != 'QUEUED') and (optimized_response.state != 'RUNNING'):
    break

  with client.prompt.with_streaming_response.stream_messages(
      job_id=prompt_optimization_status.job_id, timeout=None) as response:
    for line in response.iter_lines():
          print(line)

optimized = json.dumps(optimized_response.optimized_prompt_messages, indent=2)
display(optimized)

## Save the new system prompt template

It's convenient to stash this template for use later.

In [ ]:
import json
from pathlib import Path
from google.colab import files

filename = 'aesop_ai_dspy_prompt.json.jinja'
Path(filename).write_text(optimized)
files.download(filename)

#Run inference with updated prompt

The underlying library is https://dspy.ai/, which emits a prompt in templated form.  It's a relatively straightforward Jinja2 template.

In [ ]:
from jinja2 import Template
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
from litellm import completion
import os
import re
from google.colab import userdata

os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

prompt_template = Template(optimized)
result_extractor = re.compile(r".*\[\[ ## response ## \]\](.*)\[\[ ## completed ## \]\]", re.DOTALL)

def generate(prompt: str, pbar) -> str:
  messages = json.loads(prompt_template.render(input=prompt))
  result = completion(model="gemini/gemini-1.5-flash-8b-latest",
                      messages=messages).choices[0].message.content

  pbar.update(1)
  return result_extractor.match(result).group(1)

def do_inference():
  futures = []
  pbar = tqdm(total=len(df))
  with ThreadPoolExecutor(max_workers=4) as executor:
    for index, row in df.iterrows():
      futures.append(executor.submit(generate, row["input"], pbar))
  return [future.result() for future in futures]

df["output"] = do_inference()
df

# Save the updated dataset

Feel free to compare results from before with results after prompt optimization.  Save the updated dataset for later

In [ ]:
from google.colab import files

filename = "aesop_ai_examples_optimized.parquet"
df.to_parquet(filename)
files.download(filename)

## Next Steps

Now that you have an improved prompt using an **uncalibrated** contract, let's try **calibrating** the contract!  This should help it more closely align with what you actually value.  Proceed on to the [Contract Calibration](https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/colabs/Contract_Calibration.ipynb) colab to do this.